! nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd

## 패스 수정 부탁 드립니다.

In [2]:
### 데이터 불러오기 (정제 안 된 것)
PATH = '..'
# PATH = '/content/drive/MyDrive/Github/Ryles_Practice'
train_pos = pd.read_csv(PATH + "/DATA/open/train_pos.csv")
test_pos = pd.read_csv(PATH + "/DATA/open/test_pos.csv")

In [3]:
train_pos = train_pos.fillna("없음")
test_pos = test_pos.fillna("없음")

In [4]:
train_name_key = [str(sent) for sent in train_pos['과제명'] + " " + train_pos['요약문_한글키워드']]
train_goal = [str(sent) for sent in train_pos['요약문_연구목표']]
train_cont = [str(sent) for sent in train_pos['요약문_연구내용']]
train_effect = [str(sent) for sent in train_pos['요약문_기대효과']]
test_name_key = [str(sent) for sent in test_pos['과제명'] + " " + test_pos['요약문_한글키워드']]
test_goal = [str(sent) for sent in test_pos['요약문_연구목표']]
test_cont = [str(sent) for sent in test_pos['요약문_연구내용']]
test_effect = [str(sent) for sent in test_pos['요약문_기대효과']]
train_label = [int(str(answer)) for answer in train_pos['label']]

In [5]:
train_concat = [str(name)+" "+str(key)+" "+" ".join(str(goal).split()[:100])+" "+" ".join(str(effect).split()[:100])+" "+" ".join(str(cont).split()[:100]) for name, key, goal, effect, cont in zip(train_pos['과제명'], train_pos['요약문_한글키워드'], train_pos['요약문_연구목표'], train_pos['요약문_기대효과'], train_pos['요약문_연구내용'])]
test_concat = [str(name)+" "+str(key)+" "+" ".join(str(goal).split()[:100])+" "+" ".join(str(effect).split()[:100])+" "+" ".join(str(cont).split()[:100]) for name, key, goal, effect, cont in zip(test_pos['과제명'], test_pos['요약문_한글키워드'], test_pos['요약문_연구목표'], test_pos['요약문_기대효과'], test_pos['요약문_연구내용'])]
train_label = train_pos['label']

In [6]:
from sklearn.model_selection import train_test_split
train_data, valid_data, train_label, valid_label = train_test_split(train_name_key, train_label, test_size=0.1, random_state=42)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils.fixes import loguniform
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

In [8]:
text_clf = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf", MultinomialNB())])
# text_clf = text_clf.fit(train_concat, train_label)

distribution_lr = loguniform(0.0001, 10.0)

params_clf = {"vect__ngram_range":[(1,2), (1,3), (1,5), (1,7)],
              "clf__alpha" : distribution_lr}

nb_clf = RandomizedSearchCV(text_clf, params_clf, n_iter=50, n_jobs=-1, verbose=2, cv=10, random_state = 42)
nb_clf = nb_clf.fit(train_data, train_label)

print("Best score: {}".format(nb_clf.best_score_))
nb_clf_best = nb_clf.best_estimator_

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best score: 0.9203304882351256


In [9]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf' , TfidfTransformer()),
                    ('clf' , BernoulliNB())])

distribution_bnb = loguniform(0.0001, 10.0)

params_clf = {"vect__ngram_range" : [(1,2),(1,3),(1,5),(1,7)],
                  "clf__alpha" : distribution_bnb,
                  "clf__binarize" : distribution_bnb,
              }

bn_clf = RandomizedSearchCV(text_clf, params_clf, n_iter = 50, cv = 10, random_state = 42, verbose = 2, n_jobs = -1)
bn_clf = bn_clf.fit(train_data, train_label)

print("Best score: {}".format(bn_clf.best_score_))
bn_clf_best = bn_clf.best_estimator_

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best score: 0.9147463509363941


In [17]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression())])

distribution_lr = loguniform(0.0001, 10.0)

params_clf = {"vect__ngram_range" : [(1,2),(1,3),(1,5),(1,7)],
                 "clf__C" : distribution_lr,
                 "clf__intercept_scaling" : distribution_lr,
#                  "clf__solver" : ['newton-cg', 'lbfgs', 'liblinear']
              }

lr_clf = RandomizedSearchCV(text_clf, params_clf, n_iter = 50, cv = 5, random_state = 42, verbose = 2, n_jobs = -5)
lr_clf = lr_clf.fit(train_data, train_label)

print("Best score: {}".format(lr_clf.best_score_))
lr_clf_best = lr_clf.best_estimator_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


exception calling callback for <Future at 0x7fb6e8e6d9d0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/Users/

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGSEGV(-11)}

## 패스 수정 부탁 드립니다.

In [ ]:
import pickle
# pickle.dump(nb_clf_best, open('../nb_clf_nk_model.sav', 'wb'))
# pickle.dump(bn_clf_best, open('../bn_clf_nk_model.sav', 'wb'))
pickle.dump(lr_clf_best, open('../lr_clf_nk_model.sav', 'wb'))

In [18]:
nb_clf_best_pred = nb_clf_best.predict(valid_data)
print('NaiveBaise')
print('잘못 분류된 개수: %d' % (valid_label != nb_clf_best_pred).sum())
print('정확도: %.3f' % accuracy_score(valid_label, nb_clf_best_pred))
bn_clf_best_pred = bn_clf_best.predict(valid_data)
print('BernoisBaise')
print('잘못 분류된 개수: %d' % (valid_label != bn_clf_best_pred).sum())
print('정확도: %.3f' % accuracy_score(valid_label, bn_clf_best_pred))
# lr_clf_best_pred = lr_clf_best.predict(valid_data)
# print('LogisticRegression')
# print('잘못 분류된 개수: %d' % (valid_label != lr_clf_best_pred).sum())
# print('정확도: %.3f' % accuracy_score(valid_label, lr_clf_best_pred))

NaiveBaise
잘못 분류된 개수: 1251
정확도: 0.928
BernoisBaise
잘못 분류된 개수: 1327
정확도: 0.924


NameError: name 'lr_clf_best' is not defined

In [20]:
from sklearn.ensemble import StackingClassifier

# , ('lr', lr_clf_best)]
stack = StackingClassifier(estimators = [('nb', nb_clf_best), ('bn', bn_clf_best)], n_jobs=-1)

stack.fit(train_data, train_label)

/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('nb',
                                Pipeline(steps=[('vect',
                                                 CountVectorizer(ngram_range=(1,
                                                                              7))),
                                                ('tfidf', TfidfTransformer()),
                                                ('clf',
                                                 MultinomialNB(alpha=0.00010090061869151561))])),
                               ('bn',
                                Pipeline(steps=[('vect',
                                                 CountVectorizer(ngram_range=(1,
                                                                              5))),
                                                ('tfidf', TfidfTransformer()),
                                                ('clf',
                                                 BernoulliNB(alpha=0.00010847546640130737,
                     

## 패스 수정 부탁 드립니다.

In [ ]:
pickle.dump(stack, open('../stack_nk_nb_bn_model.sav', 'wb'))

In [ ]:
pred = stack.predict(test_name_key)

In [ ]:
sum(pred!=0)

In [ ]:
sample_submission=pd.read_csv('../DATA/open/sample_submission.csv')
sample_submission['label']=pred
sample_submission.to_csv('../DATA/Result/NB_BN_stack.csv', index=False)